In [ ]:
# data - https://www.kaggle.com/competitions/fake-news/data?select=test.csv

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)
y=df['label']

In [6]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [7]:
import tensorflow as tf
tf.__version__

'2.13.0'

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [9]:
### Vocabulary size
voc_size=5000

In [10]:
messages=X.copy()

In [11]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [17]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[914, 4664, 3184, 3164, 3830, 832, 886, 623, 1401, 3109],
 [3108, 2926, 4804, 313, 724, 912, 4631],
 [2536, 4275, 4809, 4276],
 [2909, 134, 4946, 1824, 2081, 3262],
 [3180, 724, 317, 4983, 4750, 4417, 724, 2229, 4593, 2019],
 [280,
  4277,
  1394,
  2451,
  2700,
  3184,
  1666,
  958,
  1936,
  3301,
  1330,
  2888,
  1446,
  2247,
  4631],
 [4982, 2364, 447, 2464, 3991, 2738, 4453, 883, 751, 1731, 2774],
 [2954, 2000, 3640, 2884, 4991, 1660, 3184, 921, 751, 1731, 2774],
 [1924, 3462, 693, 2386, 2911, 1082, 4412, 3486, 3184, 2850],
 [4391, 646, 4209, 4679, 1967, 3531, 4378, 2020],
 [1697, 95, 3062, 377, 3145, 297, 4380, 893, 947, 1051, 1325],
 [1824, 2238, 3830, 1082, 3184, 4991],
 [3929, 1052, 1414, 685, 3662, 515, 1025, 149, 2972],
 [3462, 3367, 3055, 718, 1332, 2953, 2294, 751, 1731, 2774],
 [326, 2244, 2297, 981, 4408, 751, 1731, 2774],
 [1456, 4249, 1388, 926, 1731, 3573, 1513, 3617, 4963, 3608],
 [512, 1418, 2926],
 [712, 3554, 1263, 3798, 3184, 4362, 4914, 4631],
 [3237, 725, 

## Embedding Representation

In [18]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  623 1401 3109]
 [   0    0    0 ...  724  912 4631]
 [   0    0    0 ... 4275 4809 4276]
 ...
 [   0    0    0 ...  751 1731 2774]
 [   0    0    0 ...  734 2000 4031]
 [   0    0    0 ... 3274  840  511]]


In [19]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  914,
       4664, 3184, 3164, 3830,  832,  886,  623, 1401, 3109])

In [20]:
## Creating model (lstm)
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [21]:
## Creating model(bidirectional lstm)
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [22]:
len(embedded_docs),y.shape

(18285, (18285,))

In [23]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [24]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [26]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 21s 66ms/step - loss: 0.3048 - accuracy: 0.8555 - val_loss: 0.2033 - val_accuracy: 0.9118
Epoch 2/10
192/192 [==============================] - 11s 56ms/step - loss: 0.1404 - accuracy: 0.9449 - val_loss: 0.2276 - val_accuracy: 0.9173
Epoch 3/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0931 - accuracy: 0.9647 - val_loss: 0.2380 - val_accuracy: 0.9195
Epoch 4/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0599 - accuracy: 0.9798 - val_loss: 0.2438 - val_accuracy: 0.9089
Epoch 5/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0364 - accuracy: 0.9870 - val_loss: 0.3870 - val_accuracy: 0.8998
Epoch 6/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0208 - accuracy: 0.9944 - val_loss: 0.4043 - val_accuracy: 0.9079
Epoch 7/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0184 - accuracy: 0.9947 - val_loss: 0.5269 - val_accuracy:

## Performance Metrics And Accuracy

In [44]:
y_pred1 = model1.predict(X_test)
y_pred1 = (y_pred1 > 0.5).astype(int)

189/189 [==============================] - 7s 23ms/step


In [45]:
y_pred1

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [46]:
from sklearn.metrics import confusion_matrix

In [47]:
confusion_matrix(y_test,y_pred1)

array([[3101,  318],
       [ 242, 2374]], dtype=int64)

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9072079536039768

In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

